In [13]:
import os
os.environ["LVMCORE_DIR"] = "/Users/droryn/prog/lvm/lvmcore"
os.environ["LVM_MASTER_DIR"] = "/Users/droryn/work/LVM/data/sas/sdsswork/lvm/sandbox/calib"
os.environ["SAS_BASE_DIR"] = "/Users/droryn/work/LVM/data/sas"
os.environ["LVM_DATA_S"] = "/Users/droryn/work/LVM/data/sas/sdsswork/data/lvm/lco"
os.environ["LVM_SPECTRO_REDUX"] = "/Users/droryn/work/LVM/data/redux"
ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")

import numpy as np
from astropy.table import Table
from astropy.io import fits

from lvmdrp import path, __version__ as drpver
from lvmdrp.utils import metadata as md
from lvmdrp.main import fibermap

In [21]:
from lvmdrp.functions import imageMethod as image_tasks
from lvmdrp.main import get_master_mjd
from lvmdrp.core.image import reject_cosmics

MJD = 60356

sci_metadata = md.get_metadata(mjd=MJD, tileid=1039949).sort_values(["expnum", "camera"])

arc_lamps = {"b": "hgne", "r": "neon", "z": "neon"}

master_mjd = get_master_mjd(MJD)

def f():
    spec_paths = []
    for sci in sci_metadata.to_dict("records"):
        sci_camera = sci["camera"]

        # define sci paths
        sci_path = path.full("lvm_raw", camspec=sci["camera"], **sci)
        psci_path = path.full("lvm_anc", drpver=drpver, kind="p", imagetype=sci["imagetyp"], **sci)
        dsci_path = path.full("lvm_anc", drpver=drpver, kind="d", imagetype=sci["imagetyp"], **sci)
        xsci_path = path.full("lvm_anc", drpver=drpver, kind="x", imagetype=sci["imagetyp"], **sci)
        wsci_path = path.full("lvm_anc", drpver=drpver, kind="w", imagetype=sci["imagetyp"], **sci)
        # lamps configuration per spectrograph channel
        lamps = arc_lamps[sci["camera"][0]]
        
        # define calibration frames paths
        masters_path = os.path.join(ORIG_MASTER_DIR, f"{master_mjd}")
        if masters_path is None:
            raise ValueError("LVM_MASTER_DIR environment variable is not defined")
        mpixmask_path = os.path.join(masters_path, f"lvm-mpixmask-{sci_camera}.fits")
        mbias_path = os.path.join(masters_path, f"lvm-mbias-{sci_camera}.fits")
        mdark_path = os.path.join(masters_path, f"lvm-mdark-{sci_camera}.fits")
        mpixflat_path = os.path.join(masters_path, f"lvm-mpixflat-{sci_camera}.fits")
        mtrace_path = os.path.join(masters_path, f"lvm-mtrace-{sci_camera}.fits")
        mwidth_path = os.path.join(masters_path, f"lvm-mwidth-{sci_camera}.fits")
        
        # if os.path.isfile(wsci_path):
        #     print(f"skipping {wsci_path}, file already exist")
        #     continue
        
        # preprocess frame
        image_tasks.preproc_raw_frame(in_image=sci_path, out_image=psci_path, in_mask=mpixmask_path)
        
        # detrend frame
        image_tasks.detrend_frame(in_image=psci_path, out_image=dsci_path, in_bias=mbias_path, in_dark=mdark_path, in_slitmap=Table(fibermap.data))
        
        # # extract 1d spectra
        # image_tasks.extract_spectra(in_image=dsci_path, out_rss=xsci_path, in_trace=mtrace_path, method="aperture", aperture=3)
        
        # # wavelength calibrate & resample
        # iwave, fwave = SPEC_CHANNELS[sci["camera"][0]]
        # rss_tasks.create_pixel_table(in_rss=xsci_path, out_rss=wsci_path, arc_wave=mwave_path, arc_fwhm=mlsf_path)
        # rss_tasks.resample_wavelength(in_rss=wsci_path, out_rss=wsci_path, method="linear", disp_pix=0.5, start_wave=iwave, end_wave=fwave, err_sim=10, parallel=0)
        
        # # apply fiberflat correction
        # rss_tasks.apply_fiberflat(in_rss=wsci_path, out_rss=wsci_path, in_flat=mflat_path)
        
        # # list paths for spectrograph combination
        # spec_paths.append(wsci_path)

[ERROR]: Traceback (most recent call last):
  File "/opt/miniconda/envs/lvmdrp/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/31/fxk1ql6s5bx7q3kh6kwpf8v8c5vp86/T/ipykernel_46348/3959995733.py", line 3, in <module>
    from lvmdrp.core.image import reject_cosmics
ImportError: cannot import name 'reject_cosmics' from 'lvmdrp.core.image' (/Users/droryn/prog/lvm/lvmdrp/python/lvmdrp/core/image.py)



In [31]:
import numpy
from lvmdrp.core.image import Image
from lvmdrp.core.image import LinearSelectionElement
from scipy import ndimage

def reject_cosmics(image, sigma_det=5, rlim=1.2, iterations=5, fwhm_gauss=[2.0,2.0], replace_box=[5, 5],
        replace_error=1e6, increase_radius=0, binary_closure=True,
        gain=1.0, rdnoise=1.0, bias=0.0, verbose=False, inplace=True):

    # convert all parameters to proper type
    sigma_x = fwhm_gauss[0] / 2.354
    sigma_y = fwhm_gauss[1] / 2.354
    box_x = int(replace_box[0])
    box_y = int(replace_box[1])

    # define Laplacian convolution kernal
    LA_kernel = 0.25*numpy.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]], dtype=numpy.float32)

    # Initiate image instances
    img_original = Image(data=image)
    img = Image(data=image)

    # subtract bias if applicable
    if (bias > 0.0) and verbose:
        print(f'Subtract bias level {bias:.2f} from image')
    img = img - bias
    img_original = img_original - bias

    # apply gain factor to data if applicable
    if (gain != 1.0) and verbose:
        print('  Convert image from ADUs to electrons using a gain factor of {gain:.2f}')
    img = img * gain
    img_original = img_original * gain

    # compute noise using read-noise value
    if (rdnoise > 0.0) and verbose:
        print(f'  A value of {rdnoise:.2f} is used for the electron read-out noise')
    img_original._error = numpy.sqrt((numpy.clip(img_original._data, a_min=0.0, a_max=None) + rdnoise**2))

    select = numpy.zeros(img._dim, dtype=bool)
    img_original._mask = numpy.zeros(img._dim, dtype=bool)
    img._mask = numpy.zeros(img._dim, dtype=bool)

    # start iteration
    out = img
    for i in range(iterations):
        if verbose:
            print(f'  Start iteration {i+1}')

        # create smoothed noise fromimage
        noise = out.medianImg((box_y, box_x))
        select_neg2 = noise._data <= 0
        noise.replace_subselect(select_neg2, data=0)
        noise = (noise + rdnoise ** 2).sqrt()

        sub = img.subsampleImg()  # subsample image
        conv = sub.convolveImg(LA_kernel)  # convolve subsampled image with kernel
        select_neg = conv < 0
        conv.replace_subselect(select_neg, data=0)  # replace all negative values with 0
        Lap = conv.rebin(2, 2)  # rebin the data to original resolution
        S = Lap/(noise*2)  # normalize Laplacian image by the noise
        S_prime = S-S.medianImg((5, 5))  # cleaning of the normalized Laplacian image

        # Perform additional clean using a 2D Gaussian smoothing kernel
        fine = out.convolveGaussImg(sigma_x, sigma_y, mask=True)  # convolve image with a 2D Gaussian
        fine_norm = out/fine
        select_neg = fine_norm < 0
        fine_norm.replace_subselect(select_neg, data=0)
        sub_norm = fine_norm.subsampleImg()  # subsample image
        Lap2 = sub_norm.convolveImg(LA_kernel)
        Lap2 = Lap2.rebin(2, 2)  # rebin the data to original resolution

        select = numpy.logical_or(numpy.logical_and(Lap2 > rlim, S_prime > sigma_det), select)

        if verbose:
            dim = img_original._dim
            det_pix = numpy.sum(select)
            print(f'  Total number of detected cosmics: {det_pix} out of {dim[0] * dim[1]} pixels')

        if i == iterations-1:
            img_original.replace_subselect(select, mask=True)  # set the new mask
            if increase_radius > 0:
                mask_img = Image(data=img_original._mask)
                mask_new = mask_img.convolveImg(kernel=numpy.ones((2*increase_radius+1, 2*increase_radius+1)))
                img_original._mask = mask_new
            if binary_closure:
                bmask = img_original._mask > 0
                bc_mask = numpy.zeros(bmask.shape, dtype=img_original._mask.dtype)
                for ang in [20, 45, 70, 90, 110, 135, 160]:
                    # leave out the dispersion direction (0 degrees), see DESI, Guy et al., ApJ, 2023, 165, 144
                    lse = LinearSelectionElement(11, 11, ang)
                    bc_mask = bc_mask | ndimage.binary_closing(bmask, structure=lse.se)
                img_original._mask = bc_mask
                if verbose:
                    print(f'  Total number after binary closing: {numpy.sum(bc_mask)} pixels')

            # replace possible corrput pixel with median for final output
            out = img_original.replaceMaskMedian(box_x, box_y, replace_error=replace_error)
        else:
            out.replace_subselect(select, mask=True)  # set the new mask
            out = out.replaceMaskMedian(box_x, box_y, replace_error=None)  # replace possible corrput pixel with median

    return out


In [32]:
image = numpy.random.rand(4000,4000)

%load_ext line_profiler
%lprun -f reject_cosmics reject_cosmics(image, verbose=1)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
  A value of 1.00 is used for the electron read-out noise
  Start iteration 1
  Total number of detected cosmics: 0 out of 16000000 pixels
  Start iteration 2
  Total number of detected cosmics: 0 out of 16000000 pixels
  Start iteration 3
  Total number of detected cosmics: 0 out of 16000000 pixels
  Start iteration 4
  Total number of detected cosmics: 0 out of 16000000 pixels
  Start iteration 5
  Total number of detected cosmics: 0 out of 16000000 pixels
  Total number after binary closing: 0 pixels


Timer unit: 1e-09 s

Total time: 66.9971 s
File: /var/folders/31/fxk1ql6s5bx7q3kh6kwpf8v8c5vp86/T/ipykernel_46348/3630395383.py
Function: reject_cosmics at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def reject_cosmics(image, sigma_det=5, rlim=1.2, iterations=5, fwhm_gauss=[2.0,2.0], replace_box=[5, 5],
     7                                                   replace_error=1e6, increase_radius=0, binary_closure=True,
     8                                                   gain=1.0, rdnoise=1.0, bias=0.0, verbose=False, inplace=True):
     9                                           
    10                                               # convert all parameters to proper type
    11         1       1000.0   1000.0      0.0      sigma_x = fwhm_gauss[0] / 2.354
    12         1          0.0      0.0      0.0      sigma_y = fwhm_gauss[1] / 2.354
    13         1       1000.0   1000.0      0.0      box_x = int(repla

In [36]:
import scipy.ndimage
from scipy import signal
image = numpy.random.rand(1000 ,1000)
%timeit ndimage.median_filter(image, size=(5,5))
%timeit signal.medfilt2d(image, kernel_size=(5,5))

319 ms ± 243 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
308 ms ± 6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
